In [4]:
import numpy as np
import numpy.linalg
import matplotlib.pyplot as plt
import tqdm

In [53]:
# solves Ax = b linear equation system
def jacobi_solver(A:np.ndarray,b:np.ndarray,initialValues = None ,acuraccy= 3):
    if initialValues is None:
        initialValues = np.zeros(b.shape)

    oldx = x = initialValues
    dInvers = np.linalg.inv(np.diag(A.diagonal()))
    tm = A - np.diag(A.diagonal())


    while(True):
        x = dInvers@(b-tm@x)
        if np.max(np.abs(np.round(x-oldx,acuraccy+1))) == 0:
            break
        oldx = x.copy()
    
    return x

# generates bellman linear equation system's coefficients coresponding to given grid_map and policy
def generate_bellman_eqSystem(grid_map,policy,terminal_states,epsilon)->tuple:
    A = np.diag(np.ones(grid_map.size))
    b = np.zeros(grid_map.size)
    
    for i in range(1,grid_map.shape[0]-1):
        for j in range(1,grid_map.shape[1]-1):
            if [i,j] in terminal_states:
                continue


            b[encodeIndex(i,j,grid_map)] = np.sum([policy[i,j,0]*grid_map[i+1,j] , policy[i,j,1]*grid_map[i-1,j] 
            , policy[i,j,2]*grid_map[i,j+1] , policy[i,j,3]*grid_map[i,j-1] ])



            A[encodeIndex(i,j,grid_map), encodeIndex(i,j,grid_map)] = 1
            A[encodeIndex(i,j,grid_map), encodeIndex(i+1,j,grid_map)] = -epsilon*policy[i,j,0]
            A[encodeIndex(i,j,grid_map), encodeIndex(i-1,j,grid_map)] = -epsilon*policy[i,j,1]
            A[encodeIndex(i,j,grid_map), encodeIndex(i,j+1,grid_map)] = -epsilon*policy[i,j,2]
            A[encodeIndex(i,j,grid_map), encodeIndex(i,j-1,grid_map)] = -epsilon*policy[i,j,3]


    return A,b


def encodeIndex(i,j,grid_map)->int:
    return i*grid_map.shape[1] + j

# simulates a gready agent's actions based on given vValues
def play_agent(vValues, startPos, terminalStates):
    vValues[0,:] = -np.inf
    vValues[-1,:] = -np.inf
    vValues[:,0] = -np.inf
    vValues[:,-1] = -np.inf
    

    res = np.zeros(vValues.shape)
    pos = startPos
    res[pos[0], pos[1]] = 8
    actions = [[1,0],[-1,0],[0,1],[0,-1]]

    while (pos not in terminalStates):
        

        action = actions[np.argmax([ vValues[pos[0]+1,pos[1]], vValues[pos[0]-1,pos[1]]
          , vValues[pos[0],pos[1]+1], vValues[pos[0],pos[1]-1] ])]
        print(pos,action)
        
        pos[0],pos[1] = pos[0]+action[0], pos[1]+action[1]
        res[pos[0], pos[1]] = 1
        
    return res
        

In [92]:
# each cell's value determines the reward of each action which moves the agent to this cell.
grid = np.array([
    [ 0,  0,  0,  0 ,  0,  0, 0],
    [ 0, 10, -1, -5 , -1, -1, 0],
    [ 0, -1, -1, -1 , -1, -1, 0],
    [ 0, -1, -1, -1 , -1, -1, 0],
    [ 0, -1, -1, -1 , -1 ,-1, 0],
    [ 0, -1, -1, -1 , -1, -1, 0],
    [ 0,  0,  0,  0 ,  0,  0, 0]
])


# initalizing the policy
policy = np.zeros([grid.shape[0], grid.shape[1], 4])

for i in range(1,grid.shape[0]-1):
    for j in range(1,grid.shape[1]-1):

        counter =0
        for a, action in enumerate([[1,0],[-1,0],[0,1],[0,-1]]):
            sPrime = [i+action[0], j+action[1]]

            if grid[sPrime[0],sPrime[1]] != 0:
                policy[i,j,a] = 1
                counter += 1

        policy[i,j,:] /= counter

policy= np.round(policy,2)


In [94]:
A,b = generate_bellman_eqSystem(grid,policy,[[1,1]],0.9)
vValues = jacobi_solver (A,b,acuraccy=10).reshape([7,7])
np.round(vValues,2)

array([[  0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ],
       [  0.  ,   0.  ,  -2.5 ,  -7.49, -10.35,  -9.97,   0.  ],
       [  0.  ,  -0.62,  -5.38,  -9.04,  -9.62,  -9.59,   0.  ],
       [  0.  ,  -5.58,  -7.3 ,  -8.78,  -9.33,  -9.36,   0.  ],
       [  0.  ,  -7.55,  -8.27,  -8.91,  -9.24,  -9.27,   0.  ],
       [  0.  ,  -8.23,  -8.53,  -8.88,  -9.13,  -9.28,   0.  ],
       [  0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ]])

In [95]:
play_agent(vValues,startPos=[1,5],terminalStates=[[1,1]])

[1, 5] [1, 0]
[2, 5] [1, 0]
[3, 5] [1, 0]
[4, 5] [0, -1]
[4, 4] [0, -1]
[4, 3] [0, -1]
[4, 2] [-1, 0]
[3, 2] [-1, 0]
[2, 2] [0, -1]
[2, 1] [-1, 0]


array([[0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 8., 0.],
       [0., 1., 1., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0., 1., 0.],
       [0., 0., 1., 1., 1., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.]])